In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys

import time

import tensorflow as tf

In [47]:
import collections
import random
import urllib
import zipfile


In [162]:
# training parameters
learning_rate = 0.1
batch_size = 128
num_steps = 3000000
display_step = 10000
eval_step = 200000

In [163]:
# evaluation parameters
eval_words = ['five', 'of', 'going', 'hardware', 'american', 'britain']

In [164]:
# Woed2Vec parameters
embedding_size = 200
max_vocabulary_size = 5000 
min_occurrence = 10 # Remove all words that does not appears at least n times
skip_window = 3 # How many words to consider left and right
num_skips = 2 # How many times to reuse an input to generate a label 从整个窗口中选取多少个不同的词
num_sampled = 64 # Number of negative examples to sample

In [165]:
# url = 'http://mattmahoney.net/dc/text8.zip'
# data_path = 'text8.zip'
# if not os.path.exists(data_path):
#     print("Downloading the dataset... (It may take some time)")
#     filename, _ = urllib.request.urlretrieve(url, data_path)
#     print("Done!")
# Unzip the dataset file. Text has already been processed
data_path = 'C:/Users/Crow/Desktop/text8.zip'
with zipfile.ZipFile(data_path) as f:
    text_words = f.read(f.namelist()[0]).lower().split()

In [166]:
text_words[0][0:5]

b'anarc'

In [167]:
len(text_words)

17005207

In [168]:
# Build the dictionary and replace rare words with UNK token
count = [('UNK', -1)]
count.extend(collections.Counter(text_words).most_common(max_vocabulary_size - 1))
for i in range(len(count) - 1,-1,-1):
    if count[i][1] < min_occurrence:
        count.pop(i)
    else:
        break
        
vocabulary_size = len(count)
word2id = dict()
for i, (word, _) in enumerate(count):
    word2id[word] = i

data = list()
unk_count = 0
for word in text_words:
    index = word2id.get(word, 0)
    if index == 0:
        unk_count += 1
    data.append(index)
count[0] = ('UNK', unk_count)
id2word = dict(zip(word2id.values(), word2id.keys()))

print("Words count:", len(text_words))
print("Unique words:", len(set(text_words)))
print("Vocabulary size:", vocabulary_size)
print("Most common words:", count[:10])

Words count: 17005207
Unique words: 253854
Vocabulary size: 5000
Most common words: [('UNK', 2735459), (b'the', 1061396), (b'of', 593677), (b'and', 416629), (b'one', 411764), (b'in', 372201), (b'a', 325873), (b'to', 316376), (b'zero', 264975), (b'nine', 250430)]


In [169]:
data_index = 0
# generate training batch for the skip-gram model
def next_batch(batch_size, num_skips, skip_window):
    global data_index # 为定义在函数外的变量赋值
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch  = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    
    # get window size (words left and right + current one)
    span = 2 * skip_window + 1
    # 设置缓冲区最大程度保留span元素，是一种用于采样的移动窗口
    buffer = collections.deque(maxlen=span) # double-ended queue 双边队列 
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            buffer.extend(data[0:span])
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    # backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [170]:
X = tf.placeholder(tf.int32, shape=[None]) # input data
Y = tf.placeholder(tf.int32, shape=[None, 1]) # input label

# ensure the following ops & var are assigned on cpu
# 有些操作在GPU上不兼容
with tf.device('/cpu:0'):
    # create the embedding variable (each row represent a word embedding vector)
    embedding = tf.Variable(tf.random_normal([vocabulary_size, embedding_size]))
    # lookup the corresponding embedding vectors for each sample in X
    X_embed = tf.nn.embedding_lookup(embedding, X)
    
    # construct the variables for the NCE loss
    # NCE（Noise Contrastive Estimation，噪声对比估计，的速度更快，可以作为替代方案。这个方法不是用上下文单词相对于词汇表中所有可能的上下文单词的概率，而是随机抽样 2-20 个可能的上下文单词，并仅从这些单词中评估概率。该方法可用于训练模型，且可大大加快训练进程
    nce_weight = tf.Variable(tf.random_normal([vocabulary_size, embedding_size]))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
    
# compute the average NCE loss for the batch
loss_op = tf.reduce_mean(
    tf.nn.nce_loss(weights=nce_weight,
                   biases=nce_biases,
                   labels=Y,
                   inputs=X_embed,
                   num_sampled=num_sampled,
                   num_classes=vocabulary_size
                  )
)

# define the optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op = optimizer.minimize(loss_op)

# evaluation
# compute the cosine similarity between input data embedding and every embedding vectors
X_embed_norm = X_embed / tf.sqrt(tf.reduce_sum(tf.square(X_embed)))
embedding_norm = embedding / tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keepdims=True))
cosine_sim_op = tf.matmul(X_embed_norm, embedding_norm, transpose_b=True) # transpose_b: 如果为真, b则在进行乘法计算前进行转置


In [175]:
# initialize the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    
    # run the initializes
    sess.run(init)
    
    # testing data
    x_test = np.array([word2id[w] for w in eval_words])
    
    average_loss = 0
    
    for step in range(1, num_steps + 1):
        # get a new batch of data
        batch_x, batch_y = next_batch(batch_size, num_skips, skip_window)
        # run trainingop
        _, loss = sess.run([train_op, loss_op], feed_dict = {X:batch_x, Y:batch_y})
        average_loss += loss
        
        if step % display_step == 0 or step == 1:
            if step > 1:
                average_loss /= display_step
            print('step ' + str(step) + ', average loss = ' + '{:.4f}'.format(average_loss))
            average_loss = 0
        
        # evaluation 
        if step % eval_step == 0 or step == 1:
            print('evaluation:')
            sim = sess.run(cosine_sim_op, feed_dict={X:x_test})
            for i in range(len(eval_words)):
                top_k = 8 # number of nearest neughbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = '"%s" nearest neighbors:' % eval_words[i]
                for k in range(top_k):
                    log_str = '%s %s,' % (log_str, id2word[nearest[k]])
                print(log_str)
            

step 1, average loss = 466.9443
evaluation:
"b'five'" nearest neighbors: b'van', b'frequency', b'kilometers', b'career', b'commission', b'benjamin', b'literally', b'captain',
"b'of'" nearest neighbors: b'yards', b'group', b'be', b'evolved', b'member', b'carrying', b'supposed', b'california',
"b'going'" nearest neighbors: b'charge', b'cook', b'chose', b'python', b'advances', b'dollar', b'stanford', b'attempted',
"b'hardware'" nearest neighbors: b'larry', b'single', b'operations', b'african', b'greatly', b'cutting', b'appearances', b'iron',
"b'american'" nearest neighbors: b'fighting', b'ex', b'combat', b'driver', b'addresses', b'pope', b'husband', b'kennedy',
"b'britain'" nearest neighbors: b'affect', b'phenomenon', b'velocity', b'controls', b'effort', b'hungarian', b'simple', b'swiss',
step 10000, average loss = 72.5968
step 20000, average loss = 18.7944
step 30000, average loss = 12.3687
step 40000, average loss = 10.1661
step 50000, average loss = 8.7488
step 60000, average loss = 8.

step 1210000, average loss = 5.0358
step 1220000, average loss = 5.0789
step 1230000, average loss = 5.0827
step 1240000, average loss = 5.0771
step 1250000, average loss = 5.0730
step 1260000, average loss = 5.0807
step 1270000, average loss = 4.8143
step 1280000, average loss = 5.0216
step 1290000, average loss = 5.0518
step 1300000, average loss = 5.0147
step 1310000, average loss = 5.0034
step 1320000, average loss = 5.0094
step 1330000, average loss = 5.0193
step 1340000, average loss = 5.0213
step 1350000, average loss = 4.9194
step 1360000, average loss = 4.9762
step 1370000, average loss = 5.0228
step 1380000, average loss = 5.0331
step 1390000, average loss = 5.0350
step 1400000, average loss = 5.0200
evaluation:
"b'five'" nearest neighbors: b'four', b'three', b'six', b'seven', b'eight', b'two', b'zero', b'one',
"b'of'" nearest neighbors: b'and', b'for', b'in', b'including', b'with', b'from', UNK, b'while',
"b'going'" nearest neighbors: b'back', b'due', b'game', b'just', b'bec

step 2410000, average loss = 4.7044
step 2420000, average loss = 4.7895
step 2430000, average loss = 4.7952
step 2440000, average loss = 4.8243
step 2450000, average loss = 4.7992
step 2460000, average loss = 4.8165
step 2470000, average loss = 4.7753
step 2480000, average loss = 4.8052
step 2490000, average loss = 4.8171
step 2500000, average loss = 4.7747
step 2510000, average loss = 4.8119
step 2520000, average loss = 4.7716
step 2530000, average loss = 4.7684
step 2540000, average loss = 4.7589
step 2550000, average loss = 4.7839
step 2560000, average loss = 4.7986
step 2570000, average loss = 4.7927
step 2580000, average loss = 4.8047
step 2590000, average loss = 4.7930
step 2600000, average loss = 4.5722
evaluation:
"b'five'" nearest neighbors: b'four', b'three', b'six', b'seven', b'eight', b'two', b'nine', b'zero',
"b'of'" nearest neighbors: b'following', b'for', b'and', b'from', b'in', b'while', b'under', b'including',
"b'going'" nearest neighbors: b'back', b'again', b'gave', b